In [1]:
import os
import pandas as pd
import sqlite3

In [2]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_logician/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_logician/')

In [4]:
conn = sqlite3.connect('data/external/chemphopro.db')

In [5]:
# data frame with "where...." adding SQL Query parameters
protein = pd.read_sql_query('SELECT * FROM protein WHERE expressed_in IS NOT NULL ;', conn)

# explode 'expressed_in' column into one row per cell line 
protein = protein.assign(expressed_in=protein['expressed_in'].str.split(',')).explode('expressed_in')

# rename cell lines
protein.loc[protein['expressed_in'].str.contains('MCF'), 'expressed_in'] = 'MCF7'
protein.loc[protein['expressed_in'].str.contains('HL'), 'expressed_in'] = 'HL60'
protein.loc[protein['expressed_in'].str.contains('NTERA'), 'expressed_in'] = 'NTERA2'

# add missing value
protein.kinase_name.fillna('PRAG1', inplace=True)

In [6]:
protein

,uniprot_name,kinase_name,expressed_in,uniprot_id,gene_name,gene_synonyms,description,families,length,sequence
0,NRBP_HUMAN,NRBP1,MCF7,Q9UHY1,NRBP1,NRBP1 BCON3 NRBP,Nuclear receptor-binding protein,"Protein kinase superfamily, Ser/Thr protein ki...",535,MSEGESQTVLSSGSDPKVESSSSAPGLTSVSPPVTSTTSAASPEEE...
0,NRBP_HUMAN,NRBP1,HL60,Q9UHY1,NRBP1,NRBP1 BCON3 NRBP,Nuclear receptor-binding protein,"Protein kinase superfamily, Ser/Thr protein ki...",535,MSEGESQTVLSSGSDPKVESSSSAPGLTSVSPPVTSTTSAASPEEE...
0,NRBP_HUMAN,NRBP1,NTERA2,Q9UHY1,NRBP1,NRBP1 BCON3 NRBP,Nuclear receptor-binding protein,"Protein kinase superfamily, Ser/Thr protein ki...",535,MSEGESQTVLSSGSDPKVESSSSAPGLTSVSPPVTSTTSAASPEEE...
1,PAK4_HUMAN,PAK4,MCF7,O96013,PAK4,PAK4 KIAA1142,Serine/threonine-protein kinase PAK 4 (EC 2.7....,"Protein kinase superfamily, STE Ser/Thr protei...",591,MFGKRKKRVEISAPSNFEHRVHTGFDQHEQKFTGLPRQWQSLIEES...
1,PAK4_HUMAN,PAK4,HL60,O96013,PAK4,PAK4 KIAA1142,Serine/threonine-protein kinase PAK 4 (EC 2.7....,"Protein kinase superfamily, STE Ser/Thr protei...",591,MFGKRKKRVEISAPSNFEHRVHTGFDQHEQKFTGLPRQWQSLIEES...
...,...,...,...,...,...,...,...,...,...,...
271,TTK_HUMAN,TTK,MCF7,P33981,TTK,TTK MPS1 MPS1L1,Dual specificity protein kinase TTK (EC 2.7.12...,"Protein kinase superfamily, Ser/Thr protein ki...",857,MESEDLSGRELTIDSIMNKVRDIKNKFKNEDLTDELSLNKISADTT...
271,TTK_HUMAN,TTK,HL60,P33981,TTK,TTK MPS1 MPS1L1,Dual specificity protein kinase TTK (EC 2.7.12...,"Protein kinase superfamily, Ser/Thr protein ki...",857,MESEDLSGRELTIDSIMNKVRDIKNKFKNEDLTDELSLNKISADTT...
271,TTK_HUMAN,TTK,NTERA2,P33981,TTK,TTK MPS1 MPS1L1,Dual specificity protein kinase TTK (EC 2.7.12...,"Protein kinase superfamily, Ser/Thr protein ki...",857,MESEDLSGRELTIDSIMNKVRDIKNKFKNEDLTDELSLNKISADTT...
272,ULK3_HUMAN,ULK3,MCF7,Q6PHR2,ULK3,ULK3,Serine/threonine-protein kinase ULK3 (EC 2.7.1...,"Protein kinase superfamily, Ser/Thr protein ki...",472,MAGPGWGPPRLDGFILTERLGSGTYATVYKAYAKKDTREVVAIKCV...


In [7]:
# export df to csv
protein.to_csv('data/processed/facts/expressed_in.csv', index=False)

In [8]:
# write Prolog file
with open('models/facts/expressed_in.pl', 'w') as file:
    for index, row in protein.iterrows():
        var1 = "expressed_in('{}', '{}').".format(row['kinase_name'], row['expressed_in'])
        file.write(var1 + '\n')